In [ ]:
# In questoo notebook carico tutto il dataset pre-processato in un dataframe pyspark
# successivamente vado a creare un file per ogni utente con tutte le righe(email) dove esso compare come mittente o destinatario

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# Install pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=a7f5a797e3f8fad1e3a1cdad086deef48a1701ed08cc9fc3871de4007183f426
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
# Carico tutti i dataset dentro un dataframe pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StringType
import pandas as pd
import numpy as np
import os

# 1) Create a SparkSession
spark = SparkSession.builder.appName("CSVtoCHUNK").config("spark.driver.memory", "16g").getOrCreate()

# 2) spark dataframe
df_email_spark = None

folder_path = '/content/gdrive/MyDrive/processati_email'

i=0

for filename in os.listdir(folder_path):

    if filename.startswith('email_') and filename.endswith('.csv'):
        i=i+1
        file_path = os.path.join(folder_path, filename)

        #read the CSV files into temporary Spark DataFrames (temp_df),
        # and append them to df_email_spark using the union method
        temp_df = spark.read.csv(file_path, header=True, inferSchema=True)

        if df_email_spark is None:
            df_email_spark = temp_df
            print(i , ' FILE caricato:', filename)# , ', righe:', df_email_spark.count())
        else:
            df_email_spark = df_email_spark.union(temp_df)
            print(i , ' FILE caricato:', filename)# , ':', df_email_spark.count())


#df_email_spark.show()
#print('dataset completo: ', df_email_spark.count())


1  FILE caricato: email_60-79.csv
2  FILE caricato: email_0-19.csv
3  FILE caricato: email_20-39.csv
4  FILE caricato: email_80-99.csv
5  FILE caricato: email_40-59.csv
6  FILE caricato: email_100-119.csv
7  FILE caricato: email_120-139.csv
8  FILE caricato: email_180-199.csv
9  FILE caricato: email_160-179.csv
10  FILE caricato: email_140-159.csv
11  FILE caricato: email_220-239.csv
12  FILE caricato: email_240-259.csv
13  FILE caricato: email_200-219.csv
14  FILE caricato: email_260-279.csv
15  FILE caricato: email_280-299.csv
16  FILE caricato: email_300-319.csv
17  FILE caricato: email_340-359.csv
18  FILE caricato: email_360-379.csv
19  FILE caricato: email_320-339.csv
20  FILE caricato: email_400-419.csv
21  FILE caricato: email_380-399.csv
22  FILE caricato: email_440-459.csv
23  FILE caricato: email_420-439.csv
24  FILE caricato: email_480-500.csv
25  FILE caricato: email_460-479.csv


In [ ]:
 df_anagrafici_excel = pd.read_excel(io = "/content/gdrive/MyDrive/Colab Notebooks/Dati per POLIMI RIVISTO_OUTPUT.xlsx", engine='openpyxl', sheet_name= 'OUTPUT')

 df_ID = df_anagrafici_excel['CODICI RISPONDENTI'].unique().tolist()

 print(len(df_ID))


2041


In [ ]:
# Here I take all the rows from the whole dataset where the user apper as a sender or as a receiver and I create a new file for each user.

csv_file_path = '/content/gdrive/MyDrive/processati_email/chunked_userID/'

print(len(df_ID))
print()


for  user_id in df_ID:
    #if index< 2034 : continue
    #if index== 200 : break

    #user_id = row['user_id']
    if ( (user_id=="ND")|(user_id=="Consulenti")|(user_id=="External") ): continue
    #print(index, ' ', user_id)
    #if user_id not in ['sWEUn-hRT1SjyBqfdlTDsg', 'FgTv82gESKu4DfVY0F435w', 'XdBe3NO6T0edHWEfoKChpg', 'Lstjqws8S86YulPOxOP4Mw', '60bCgQeiQ_iQ6B6X2sRsBg', 'ZEQWsofySoOFuU3ebdxpRA', 'hPx21qzXQaSJjaBuMWrehA', 'nmSPsaCBSuOVb2y0Erm-Sw' ] : continue

    # Create the temporary DataFrame "df_temp" containing rows with matching "user_id"
    df_temp = df_email_spark.filter((F.col("source_from_header_address") == user_id) | (F.col("destination_address") == user_id))

    pandas_df_temp = df_temp.toPandas()
    csv_filename = f"{user_id}.csv"

    pandas_df_temp.to_csv(os.path.join(csv_file_path, csv_filename), header=True, index=False)

    #print(f"'{index}' DataFrame for user_id '{user_id}' saved as '{csv_filename}'.")






2041

